In [ ]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule_multigpu
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "8"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton
from ase.optimize import LBFGS

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

from ase.parallel import world

In [ ]:
dir = '/home/jsmith48/scratch/ANI-2x_retrain/ANI-2x_MDDATA/'


fname = 'GSK1107112A'

# Molecule file
molfile = dir + fname + '.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'opt_' +  fname +'.xyz'

dt = 0.4
C = 0.0001 # Optimization convergence

ntdir = '/home/jsmith48/scratch/ANI-2x_retrain/train/ANI-2x_v62518/'
cns = ntdir + 'rHCNOSFCl-5.2R_16-3.8A_a4-8.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 8

In [ ]:
#tvec = np.loadtxt(dir+'/'+fname+'.tvec',dtype=str)
#tvec = np.array(tvec[:,1:],dtype=np.float64)

In [ ]:
#tvec

In [ ]:
# Load molecule
mol = read(molfile)
#mol = read('/home/jsmith48/scratch/dipole_training/water.xyz')
#L=40.0
# mol.set_cell(([[100.0, 0, 0],
#                 [0, 35.0, 0],
#                 [0, 0, 35.0]]))

#mol.set_cell((tvec))


#mol.set_pbc((True, True, True))
mol.set_pbc((False, False, False))

# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 1)
#aens = ensemblemolecule(cns, sae, nnf, Nn, 1)

# Set ANI calculator
mol.set_calculator(ANIENS(aens))

s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

print(E)

spc = mol.get_chemical_symbols()
print(len(spc))

In [ ]:
# Optimize molecule
start_time = time.time()
dyn = LBFGS(mol)
dyn.run(fmax=C,steps=3000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)

# printf ("Aj: %i\n",Aj);
hdt.writexyzfile(optfile, pos, spc)

In [ ]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
#dyn = NVTBerendsen(mol, 1 * units.fs, 100.0, taut=1.0*1000*units.fs)
# coefficient to 0.04 atomic units.
dyn = Langevin(mol, dt * units.fs, 1.0 * units.kB, 0.04)

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  hdt.evtokcal*a.calc.stddev
    Fstddev =  hdt.evtokcal*a.calc.Fstddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + ' ' + str(stddev) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=False)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f Esig = %.3f Fsig = %.3f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev,Fstddev))
    
# Set the printer
dyn.attach(storeenergy, interval=25)

In [ ]:
start_time = time.time()
dyn.set_temperature(300. * units.kB)
dyn.run(1250000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')